In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [12]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\vamsh\\OneDrive\\Desktop\\Langchain\\Langchain\\Lang\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [13]:
wiki.name

'wikipedia'

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACKING"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")


In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriver = vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002CF89E4ADE0>, search_kwargs={})

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, "langsmith_search", "Search for information about Langchain.")
retriver_tool.name

'langsmith_search'

In [14]:
tools = [wiki, retriver_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\vamsh\\OneDrive\\Desktop\\Langchain\\Langchain\\Lang\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about Langchain.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002CFE79FB420>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002CF89E4ADE0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000002CFE7D14D60>, retriever=VectorStoreRetriev

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [23]:
from langchain import hub

prompt = hub.pull("brianxiadong/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['system'], input_types={}, partial_variables={}, template='{system}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
### Using agents to retreive from multiple documents.

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [25]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'system'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='Human

In [29]:
agent_executor.invoke({"input": "Tell me about langsmith", "system": "", "chat_history": [], "agent_scratchpad": ""})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'langsmith'}`


LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Additional resources​
Versions​
See what changed in v0.3, learn how to migrate legacy code, read up on our versioning policies, and more.
Security​
Read up on security best practices to make sure you're developing safely with LangChain.
Contributing​
Check out the developer's guide for guidelines on contributing and help getting your dev environment set up.Edit this pageWas this page helpful?NextTutorialsArchitectureGuidesTutorialsHow-to guidesConceptual guideIntegrationsAPI referenceEcosystem🦜🛠️ LangSmith🦜🕸️ LangGraphAdditional resourcesVersionsSecurityContributingCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2025 LangChain, Inc.

Introduction | 🦜️🔗 LangChain

storesWhy LangChain?Ecosystem🦜🛠️

{'input': 'Tell me about langsmith',
 'system': '',
 'chat_history': [],
 'agent_scratchpad': '',
 'output': 'LangChain is a framework designed for developing applications powered by large language models (LLMs). It simplifies every stage of the LLM application lifecycle, providing resources for versioning, security best practices, and contributing guidelines. LangChain also includes tools like LangSmith and LangGraph, and offers a comprehensive ecosystem for developers. The framework supports various migration paths and compatibility with different versions, ensuring a smooth transition for developers working with legacy code.'}